# TOPIC RANK

* This code runs TOPICRANK model for extracting keyphrases and it doesnt need any TF-IDF computation
    
* The input testdata will need complete test file, rather than sentences like supervised models.  This can be modified using line ```extractor.load_document(input='../../RQ1.1/train_data/document/test/test.txt',```

* Also to have comparable results we do fuzzy matching with sentence where we extracted, so to produce results for diifferent folds. Please change the names of test folds as per your requirement. See line ```df=pd.read_csv("../../RQ1.1/train_data/tsv/test2.tsv",delimiter="\t")```. Folds for files include **test1.tsv** and **test2.tsv**

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# this example uses TopicRank
from pke.unsupervised import TopicRank
from collections import Counter
# create a TopicRank extractor
extractor = TopicRank()

# load the content of the document, here in CoreNLP XML format
# the input language is set to English (used for the stoplist)
# normalization is set to stemming (computed with Porter's stemming algorithm)
extractor.load_document(input='../../RQ1.1/train_data/document/test/test.txt',
                        language="en",
                        normalization='stemming')

# select the keyphrase candidates, for TopicRank the longest sequences of 
# nouns and adjectives
extractor.candidate_selection(pos={'NOUN', 'PROPN', 'ADJ'})

# weight the candidates using a random walk. The threshold parameter sets the
# minimum similarity for clustering, and the method parameter defines the 
# linkage method
extractor.candidate_weighting(threshold=0.1,
                              method='average')
allkeyphrases=[]
# print the n-highest (10) scored candidates



for (keyphrase, score) in extractor.get_n_best(n=10, stemming=False):
    allkeyphrases.append(keyphrase)

df=pd.read_csv("../../RQ1.1/train_data/tsv/test2.tsv",delimiter="\t")
texts=df["text"]
texts=[i.replace(",","") for i in texts]
labels=df["label"]
labels=[0 if i==0 else 1 for i in labels]

overall_evidence=[]
removed_text=[]
for txt in texts:
    evidence=[]
    for phr in allkeyphrases :
        ##Fuzzy Matching
        if phr in txt and txt not in removed_text:
            evidence.append(1)
            removed_text.append(txt)
    if evidence==[]:
        evidence=[0]
    overall_evidence.append(evidence)

ypred=[Counter(i).most_common(1)[0][0] for i in overall_evidence]
# print(ypred)

from sklearn.metrics import classification_report,f1_score
print(f1_score(labels,ypred,average='macro'))


# TEXT RANK

* This code runs TEXTRANK model for extracting keyphrases and it doesnt need any TF-IDF computation
    
* The input testdata will need complete test file, rather than sentences like supervised models.  This can be modified using line ```extractor.load_document(input='../../RQ1.1/train_data/document/test/test.txt',```

* Also to have comparable results we do fuzzy matching with sentence where we extracted, so to produce results for diifferent folds. Please change the names of test folds as per your requirement. See line ```df=pd.read_csv("../../RQ1.1/train_data/tsv/test2.tsv",delimiter="\t")```. Folds for files include **test1.tsv** and **test2.tsv**

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# this example uses TopicRank
from pke.unsupervised import TopicRank
from collections import Counter
import pke
# this example uses TopicRank
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input='../../RQ1.1/train_data/document/test/test.txt',
                        language='en',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)


# print the n-highest (10) scored candidates


allkeyphrases=[]
# print the n-highest (10) scored candidates
for (keyphrase, score) in extractor.get_n_best(n=200, stemming=False):
    allkeyphrases.append(keyphrase)

df=pd.read_csv("../../RQ1.1/train_data/tsv/test2.tsv",delimiter="\t")
texts=df["text"]
texts=[i.replace(",","") for i in texts]
labels=df["label"]
labels=[0 if i==0 else 1 for i in labels]

overall_evidence=[]
removed_text=[]
for txt in texts:
    evidence=[]
    for phr in allkeyphrases :
         ##Fuzzy Matching
        if phr in txt and txt not in removed_text:
            evidence.append(1)
            removed_text.append(txt)
    if evidence==[]:
        evidence=[0]
    overall_evidence.append(evidence)

ypred=[Counter(i).most_common(1)[0][0] for i in overall_evidence]
print(ypred)

from sklearn.metrics import classification_report
print(classification_report(labels,ypred,digits=5))

# SINGLE RANK

* This code runs SINGLERANK model for extracting keyphrases and it doesnt need any TF-IDF computation
    
* The input testdata will need complete test file, rather than sentences like supervised models.  This can be modified using line ```extractor.load_document(input='../../RQ1.1/train_data/document/test/test.txt',```

* Also to have comparable results we do fuzzy matching with sentence where we extracted, so to produce results for diifferent folds. Please change the names of test folds as per your requirement. See line ```df=pd.read_csv("../../RQ1.1/train_data/tsv/test2.tsv",delimiter="\t")```. Folds for files include **test1.tsv** and **test2.tsv**

In [ ]:
import pke

# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a SingleRank extractor.
extractor = pke.unsupervised.SingleRank()

# 2. load the content of the document.
extractor.load_document(input='../../RQ1.1/train_data/document/test/test.txt',
                        language='en',
                        normalization=None)

# 3. select the longest sequences of nouns and adjectives as candidates.
extractor.candidate_selection(pos=pos)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk. In the graph, nodes are words of
#    certain part-of-speech (nouns and adjectives) that are connected if
#    they occur in a window of 10 words.
extractor.candidate_weighting(window=10,
                              pos=pos)

# 5. get the 10-highest scored candidates as keyphrases
allkeyphrases=[]
# print the n-highest (10) scored candidates
for (keyphrase, score) in extractor.get_n_best(n=100, stemming=False):
    allkeyphrases.append(keyphrase)

df=pd.read_csv("../train_data/tsv/test2.tsv",delimiter="\t")
texts=df["text"]
texts=[i.replace(",","") for i in texts]
labels=df["label"]
labels=[0 if i==0 else 1 for i in labels]

overall_evidence=[]
removed_text=[]
for txt in texts:
    evidence=[]
    for phr in allkeyphrases :
         ##Fuzzy Matching
        if phr in txt and txt not in removed_text:
            evidence.append(1)
            removed_text.append(txt)
    if evidence==[]:
        evidence=[0]
    overall_evidence.append(evidence)

ypred=[Counter(i).most_common(1)[0][0] for i in overall_evidence]
print(ypred)

from sklearn.metrics import classification_report
print(classification_report(labels,ypred,digits=5))

# POSITION RANK

* This code runs POSITIONRANK model for extracting keyphrases and it doesnt need any TF-IDF computation
    
* The input testdata will need complete test file, rather than sentences like supervised models.  This can be modified using line ```extractor.load_document(input='../../RQ1.1/train_data/document/test/test.txt',```

* Also to have comparable results we do fuzzy matching with sentence where we extracted, so to produce results for diifferent folds. Please change the names of test folds as per your requirement. See line ```df=pd.read_csv("../../RQ1.1/train_data/tsv/test2.tsv",delimiter="\t")```. Folds for files include **test1.tsv** and **test2.tsv**

In [ ]:
import pke

# define the valid Part-of-Speeches to occur in the graph
pos = {'NOUN', 'PROPN', 'ADJ'}

# define the grammar for selecting the keyphrase candidates
grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"

# 1. create a PositionRank extractor.
extractor = pke.unsupervised.PositionRank()

# 2. load the content of the document.
extractor.load_document(input='../../RQ1.1/train_data/document/test/test.txt',
                        language='en',
                        normalization=None)

# 3. select the noun phrases up to 3 words as keyphrase candidates.
extractor.candidate_selection(grammar=grammar,
                              maximum_word_number=5)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk biaised with the position of the words
#    in the document. In the graph, nodes are words (nouns and
#    adjectives only) that are connected if they occur in a window of
#    10 words.
extractor.candidate_weighting(window=10,
                              pos=pos)

# 5. get the 10-highest scored candidates as keyphrases
allkeyphrases=[]
# print the n-highest (10) scored candidates
for (keyphrase, score) in extractor.get_n_best(n=200, stemming=False):
    allkeyphrases.append(keyphrase)

df=pd.read_csv("../../RQ1.1/train_data/tsv/test2.tsv",delimiter="\t")
texts=df["text"]
texts=[i.replace(",","") for i in texts]
labels=df["label"]
labels=[0 if i==0 else 1 for i in labels]

overall_evidence=[]
removed_text=[]
for txt in texts:
    evidence=[]
    for phr in allkeyphrases :
         ##Fuzzy Matching
        if phr in txt and txt not in removed_text:
            evidence.append(1)
            removed_text.append(txt)
    if evidence==[]:
        evidence=[0]
    overall_evidence.append(evidence)

ypred=[Counter(i).most_common(1)[0][0] for i in overall_evidence]
print(ypred)

from sklearn.metrics import classification_report
print(classification_report(labels,ypred,digits=5))

# MULTIPARTITE RANKING

* This code runs MULTIPARTITERANK model for extracting keyphrases and it doesnt need any TF-IDF computation
    
* The input testdata will need complete test file, rather than sentences like supervised models.  This can be modified using line ```extractor.load_document(input='../../RQ1.1/train_data/document/test/test.txt',```

* Also to have comparable results we do fuzzy matching with sentence where we extracted, so to produce results for diifferent folds. Please change the names of test folds as per your requirement. See line ```df=pd.read_csv("../../RQ1.1/train_data/tsv/test2.tsv",delimiter="\t")```. Folds for files include **test1.tsv** and **test2.tsv**

In [ ]:
import pke
import pandas as pd
import string
from nltk.corpus import stopwords

# 1. create a MultipartiteRank extractor.
extractor = pke.unsupervised.MultipartiteRank()

# 2. load the content of the document.
extractor.load_document(input='../../RQ1.1/train_data/document/test/test.txt')

# 3. select the longest sequences of nouns and adjectives, that do
#    not contain punctuation marks or stopwords as candidates.
pos = {'NOUN', 'PROPN', 'ADJ'}
stoplist = list(string.punctuation)
stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
stoplist += stopwords.words('english')
extractor.candidate_selection(pos=pos, stoplist=stoplist)

# 4. build the Multipartite graph and rank candidates using random walk,
#    alpha controls the weight adjustment mechanism, see TopicRank for
#    threshold/method parameters.
extractor.candidate_weighting(alpha=1.1,
                              threshold=0.74,
                              method='average')

# 5. get the 10-highest scored candidates as keyphrases
allkeyphrases=[]
# print the n-highest (10) scored candidates
for (keyphrase, score) in extractor.get_n_best(n=10, stemming=False):
    allkeyphrases.append(keyphrase)

df=pd.read_csv("../../RQ1.1/train_data/tsv/test1.tsv",delimiter="\t")
texts=df["text"]
texts=[i.replace(",","") for i in texts]
labels=df["label"]
labels=[0 if i==0 else 1 for i in labels]

overall_evidence=[]
removed_text=[]
for txt in texts:
    evidence=[]
    for phr in allkeyphrases :
         ##Fuzzy Matching
        if phr in txt and txt not in removed_text:
            evidence.append(1)
            removed_text.append(txt)
    if evidence==[]:
        evidence=[0]
    overall_evidence.append(evidence)

ypred=[Counter(i).most_common(1)[0][0] for i in overall_evidence]
print(ypred)

from sklearn.metrics import classification_report
print(classification_report(labels,ypred,digits=5))